<a href="https://colab.research.google.com/github/nobottle/deeplearning_pratice/blob/main/%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C%EB%B2%95_%ED%99%9C%EC%84%B1%ED%99%94%ED%95%A8%EC%88%98_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from common.functions import *
from common.util import im2col, col2im


class Relu:
    def __init__(self):
        self.mask = None #ReLu클래스는 mask라는 인스턴스 변수를 가짐, masks는 True/False로 구성된 넘파이 배열 순전파x>0이면 False,x<=0 True를 반환

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
'''데이터 x=[1,-2,3,-4]가 Relu층에 들어왔다고 하면 Relu함수의 정의가 음수는 0으로 만들고 양수는 그대로 두는거니까.
[1,0,3,0]이 나오도록 코딩을 해야 겠죠.
교재 코드에서는 트릭을 쓰는데 x <= 0으로 부울 값들을 만들어요.
이는 [1 <= 0, -2 <=0, 3 <= 0, -4 <= 0]이 되고 이는 [False, True, False, True]에요.
x를 copy해서 out을 만든 다음 이 부울 값으로 슬라이싱을 하면 True자리만 뽑혀서 [-2,-4]가 나오고 이를 0으로 두면
out은 [1,0,3,0]이 되서 원하는 값이 나와요.'''


        return dx


class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx


class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx